In [1]:
import csv
import numpy
import sys
import wfdb
from wfdb import processing

ann_str_to_num  = {
    '(AFIB\x00': 0,
    '(ASYS\x00': 1,
    '(B\x00': 2,
    '(BI\x00': 3,
    '(HGEA\x00': 4,
    '(N\x00': 5,
    '(NSR\x00': 5,
    '(NOD\x00': 6,
    '(NOISE\x00': 7,
    '(PM\x00': 8,
    '(SBR\x00': 9,
    '(SVTA\x00': 10,
    '(VER\x00': 11,
    '(VF\x00': 12,
    '(VFIB\x00': 12,
    '(VFL\x00': 13,
    '(VT\x00': 14
}


VFDB_LOCATION = './vfdb/'
QRS_LOCATION = './vfdb-qrs/'
VFDB_RECORDS = [418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428,
                429, 430, 602, 605, 607, 609, 610, 611, 612, 614, 615]

def load_qrs_i(record):
    """Returns a list with the positions of Rs that pan_tompkins returned for
       a specific record.
    """
    filepath = QRS_LOCATION + record + '_1s_i.txt'
    qrs_i = numpy.genfromtxt(filepath, delimiter=',')
    return qrs_i

def load_qrs_amp(record):
    """Returns a list with the amp values that pan_tompkins returned for
       a specific record.
    """
    filepath = QRS_LOCATION + record + '_1s_amp.txt'
    qrs_amp = numpy.genfromtxt(filepath, delimiter=',')
    return qrs_amp

def get_annotation(record):
    """Returns the annotatation object for a given record
    """
    record_path = VFDB_LOCATION + record
    ann = wfdb.rdann(record_path, 'atr')
    return ann

def get_record(record):
    """Returns the annotatation object for a given record
    """
    record_path = VFDB_LOCATION + record
    record = wfdb.rdrecord(record_path)
    return record

def get_annotation_positions(ann):
    """ Takes an annotation object and returns a list of tuples:
        [
        ('<idx if annotaion position>', 'annotations aux_label'),
        ('<idx if annotaion position>', 'annotations aux_label'),
        ...
        ('<idx if annotaion position>', 'annotations aux_label')
        ]
    """
    return list(zip(ann.sample.tolist(), ann.aux_note))

def create_qrs_annotation_mapping(qrs_i, qrs_amp, ann_pos):
    """Assign annotations to every R peak.

    :param qrs_i: list with the r positions
    :param qrs_amp: amp values in the qrs_i positions
    :param ann_pos: list of tuples with the position of every annotation

    returns a ndarray (number of R, 3) Every row will contain the
            index of the R pick, the amp of the peak and the annotation
            as number. See the dict in the beggining of this file for the
            mapping.
            [
                [index_of_R_peak, value_of_amp, annotation_id],
                ...
                [index_of_R_peak, value_of_amp, annotation_id]
            ]
    """
    # From 0 to the first annotation set the annotation to normal.
    ann_pos = [(0, '(N\x00')] + ann_pos
    # number of annotation
    len_ann_pos = len(ann_pos)
    # Number of R peaks
    len_qrs_i = len(qrs_i)
    # Initialize a zeros array for the result
    res = numpy.zeros((len_qrs_i, 3))
    # For every annotation loop the R peaks
    i = 0
    while i < len_ann_pos:
        # Take the current annotation string and its position in the siganl
        pos, ann = ann_pos[i]
        # Take the posotion of the next annotation
        ann = ann_str_to_num[ann]
        # In the case of the last annotation loop until the end of the R peaks
        # array.
        if i == len_ann_pos - 1:
            next_pos = qrs_i[qrs_i.size - 1] + 1
        else:
            next_pos, next_ann = ann_pos[i + 1]
        # For evert peak:
        x = 0
        while x < len_qrs_i:
            # If the R index is after the index of the annotation and
            # before the next annotation (ot the end of the R array)
            # set the annotation string
            if pos <= qrs_i[x] and qrs_i[x] < next_pos:
                res[x][0] = qrs_i[x]
                res[x][1] = qrs_amp[x]
                # if the annotation is NOISE a.k.a 7, set the previous
                # annotation
                if ann == 7 and pos != 0:
                    ann = ann_str_to_num[ann_pos[i - 1][1]]
                res[x][2] = ann
            x = x +1
        i = i + 1

    return res

def create_rr_annotation(annotated_r_peaks):
    """Create RR segments and annotate them.

    :param qrs_i: np.array with the r positions
    :param ann_pos: list of tuples with the position of every annotation

    returns a length(qrs_i) by 4 array which looks like this:
    [
        [start_or_RR_segment, end_or_RR_segnemnt, RR_distane (aka end_or_RR_segnemnt - start_or_RR_segment), annotation_id]
        ...
        [start_or_RR_segment, end_or_RR_segnemnt, RR_distane (aka end_or_RR_segnemnt - start_or_RR_segment), annotation_id]
    ]
    """
    res = numpy.zeros((annotated_r_peaks[:,0].size - 1, 4))
    # Set the start of the RR segments
    # Take all R indices
    all_r_idx = annotated_r_peaks[:,0]
    # Take the annotations for all Rs
    all_annots = annotated_r_peaks[:,2]

    # For the start R take from the first to the second to last
    # position 0 to -1
    start_r = all_r_idx[0:-1]
    # For the end R take from the second to the end, position 1 to end
    end_r = all_r_idx[1:all_r_idx.size]
    if start_r.size != end_r.size:
        raise Exception("start_r and end_r have diffrent sizes.")
    # The 1st column is the start R peak
    res[:,0] = start_r
    # The 2nd column is the end R peak
    res[:,1] = end_r
    # The 3rd column is the difference of end - start Rs
    res[:,2] = end_r - start_r
    # The 4th column is the annotation of the RR segment, defined by the
    # annotation of the end R.
    res[:,3] = all_annots[1:all_annots.size]

    return res

def save_rr_segment_to_csv(record_id):
    """
    Takes a record id from vfdb and writes the RR segment with its annotation
    to a csv file.

    :param record_id: The record ID eg 418
    """

    qrs_i = load_qrs_i(record_id)
    #print type(qrs_i)
    qrs_amp = load_qrs_amp(record_id)
    ann = get_annotation(record_id)
    annot_positions = get_annotation_positions(ann)
    annotated_r_peaks = create_qrs_annotation_mapping(qrs_i, qrs_amp, annot_positions)
    annotated_rr_segments = create_rr_annotation(annotated_r_peaks)
    # numpy.set_printoptions(suppress=True)
    # print annotated_rr_segments
    numpy.savetxt(QRS_LOCATION + record_id + '_RR.txt', annotated_rr_segments, delimiter=",")


In [7]:
ann_str_to_num  = {
    '(AFIB\x00': 0,
    '(ASYS\x00': 1,
    '(B\x00': 2,
    '(BI\x00': 3,
    '(HGEA\x00': 4,
    '(N\x00': 5,
    '(NSR\x00': 5,
    '(NOD\x00': 6,
    '(NOISE\x00': 7,
    '(PM\x00': 8,
    '(SBR\x00': 9,
    '(SVTA\x00': 10,
    '(VER\x00': 11,
    '(VF\x00': 12,
    '(VFIB\x00': 12,
    '(VFL\x00': 13,
    '(VT\x00': 14
}

ann_num_to_str = {}
for k, v in ann_str_to_num.items():
    if v == 5:
        k = '(N\x00'
    elif v == 12:
        k = '(VF\x00'
    ann_num_to_str[v] = k
for item in ann_num_to_str.items():
    print (item)

(0, '(AFIB\x00')
(1, '(ASYS\x00')
(2, '(B\x00')
(3, '(BI\x00')
(4, '(HGEA\x00')
(5, '(N\x00')
(6, '(NOD\x00')
(7, '(NOISE\x00')
(8, '(PM\x00')
(9, '(SBR\x00')
(10, '(SVTA\x00')
(11, '(VER\x00')
(12, '(VF\x00')
(13, '(VFL\x00')
(14, '(VT\x00')


In [9]:
print (VFDB_RECORDS)

[418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 602, 605, 607, 609, 610, 611, 612, 614, 615]


In [12]:
import numpy
print ("\nrecord \t num_of_RR")
for record in VFDB_RECORDS:
    filepath = QRS_LOCATION + str(record) + '_RR.csv'
    rr = numpy.genfromtxt(filepath, delimiter=',')
    print (record, '\t', rr[:,0].size)


record 	 num_of_RR
418 	 4300
419 	 3905
420 	 2923
421 	 4525
422 	 4072
423 	 3537
424 	 3795
425 	 2128
426 	 3774
427 	 3894
428 	 3265
429 	 3613
430 	 4695
602 	 3307
605 	 1921
607 	 1847
609 	 2214
610 	 2166
611 	 3947
612 	 2489
614 	 3329
615 	 1592


In [14]:
# Find all different labels in every annotation
import pprint

records = [str(x) for x in VFDB_RECORDS]
for record in records:
    ann = get_annotation(record)
    #print record
    #pprint.pprint(set(ann.aux_note))
    labels = set(ann.aux_note)
    print (record, labels)

418 {'(N\x00', '(VFL\x00'}
419 {'(NOISE\x00', '(N\x00', '(VFL\x00'}
420 {'(NOISE\x00', '(N\x00', '(VT\x00'}
421 {'(NOISE\x00', '(N\x00', '(VT\x00'}
422 {'(NOISE\x00', '(N\x00', '(VT\x00', '(VFIB\x00'}
423 {'(VT\x00', '(NOD\x00', '(N\x00', '(AFIB\x00', '(ASYS\x00', '(NOISE\x00'}
424 {'(NOD\x00', '(VFL\x00', '(NSR\x00', '(N\x00', '(VFIB\x00', '(ASYS\x00', '(NOISE\x00'}
425 {'(NOISE\x00', '(N\x00', '(B\x00', '(VT\x00'}
426 {'(VT\x00', '(VF\x00', '(N\x00', '(SVTA\x00', '(NOISE\x00'}
427 {'(N\x00', '(VT\x00', '(ASYS\x00'}
428 {'(NOISE\x00', '(VT\x00', '(BI\x00'}
429 {'(NOISE\x00', '(VFL\x00', '(VT\x00', '(BI\x00'}
430 {'(SBR\x00', '(VT\x00', '(BI\x00', '(VER\x00', '(VFL\x00', '(VF\x00', '(HGEA\x00', '(ASYS\x00', '(NOISE\x00'}
602 {'(VT\x00', '(PM\x00', '(N\x00', '(ASYS\x00', '(NOISE\x00'}
605 {'(NOISE\x00', '(VT\x00'}
607 {'(SVTA\x00', '(VT\x00', '(NOD\x00', '(ASYS\x00'}
609 {'(VT\x00', '(VER\x00', '(VFL\x00', '(N\x00', '(AFIB\x00', '(HGEA\x00'}
610 {'(NOISE\x00', '(N\x00', '(VT\x00', '(HGE

In [26]:
from collections import Counter
import numpy
import pprint
#import vfdb_annotate_r

records = [str(x) for x in VFDB_RECORDS]
#records = ['418']
# each segment will have 100 RR
SEGMENT_LEN = 100

for record in records:
    filepath = QRS_LOCATION + str(record) + '_RR.csv'
    rrs = numpy.genfromtxt(filepath, delimiter=',')
    rows, columns = rrs.shape
    num_segments = rows / (SEGMENT_LEN / 2)
    i = 0
    l = []
    while i < num_segments:
        start = int(i * (SEGMENT_LEN / 2))
        if i == num_segments - 1:
            end = rows - 1
        else:
            end = start + SEGMENT_LEN
        # start is the idx of the first RR of this segment
        # end is the idx of the last RR in this segment
        
        labels = rrs[start:end, 3]
        data = Counter(labels)
        most_frequent = max(data)
        if data[most_frequent] >= 0.8 * SEGMENT_LEN:
            l.append(ann_num_to_str[most_frequent])
        else:
            l.append("NO_LABEL")
                
        #print [ann_num_to_str[x] for x in list(set(labels))]            
        i += 1
    print ("record", record, "has", num_segments, "segments", Counter(l))

('record', '418', 'has', 86, 'segments', Counter({'NO_LABEL': 49, '(N\x00': 37}))
('record', '419', 'has', 78, 'segments', Counter({'(VFL\x00': 34, '(N\x00': 26, 'NO_LABEL': 18}))
('record', '420', 'has', 58, 'segments', Counter({'(N\x00': 31, '(VT\x00': 25, 'NO_LABEL': 2}))
('record', '421', 'has', 90, 'segments', Counter({'(N\x00': 57, '(VT\x00': 19, 'NO_LABEL': 14}))
('record', '422', 'has', 81, 'segments', Counter({'(N\x00': 44, '(VF\x00': 24, '(VT\x00': 10, 'NO_LABEL': 3}))
('record', '423', 'has', 70, 'segments', Counter({'(N\x00': 23, '(VT\x00': 23, '(AFIB\x00': 19, 'NO_LABEL': 5}))
('record', '424', 'has', 75, 'segments', Counter({'(N\x00': 58, 'NO_LABEL': 10, '(VF\x00': 5, '(VFL\x00': 2}))
('record', '425', 'has', 42, 'segments', Counter({'(N\x00': 27, 'NO_LABEL': 8, '(B\x00': 5, '(VT\x00': 2}))
('record', '426', 'has', 75, 'segments', Counter({'(VF\x00': 42, '(N\x00': 15, 'NO_LABEL': 14, '(VT\x00': 4}))
('record', '427', 'has', 77, 'segments', Counter({'(VT\x00': 44, '(ASYS\x

In [27]:
# Drop the first 10 RRs
# group in segments of 10 RR
# print and write in file:
# record number, [RR-0, RR-1, RR-2 ... RR-9], annotation string
# example:
# ['418', array([189., 140., 134., 135., 133.,  89.,  83., 115.,  83., 167.]), 5.0]

from collections import Counter
import numpy
import pprint
import csv

records = [str(x) for x in VFDB_RECORDS]
#records = ['418']
# each segment will have 100 RR
SEGMENT_LEN = 10
segments_fp =  QRS_LOCATION + 'segments.csv'

with open(segments_fp, 'w') as fh:
    for record in records:
        filepath = QRS_LOCATION + str(record) + '_RR.csv'
        rrs = numpy.genfromtxt(filepath, delimiter=',')
        rows, columns = rrs.shape
        #num_segments = rows / (SEGMENT_LEN / 2)
        num_segments = int(rows / SEGMENT_LEN)
        # start from the 2nd segment
        i = 1
        l = []
        while i < num_segments - 1:
            start = i * SEGMENT_LEN
            end = start + SEGMENT_LEN
            labels = rrs[start:end, 3]
            current_rrs = rrs[start:end, 2]
            data = Counter(labels)
            most_frequent = max(data)
            if data[most_frequent] == SEGMENT_LEN:
                entry = [record, current_rrs, most_frequent]
                print(entry)
                writer = csv.writer(fh)
                writer.writerow(entry)
                
            i += 1

['418', array([189., 140., 134., 135., 133.,  89.,  83., 115.,  83., 167.]), 5.0]
['418', array([138.,  82., 186., 136.,  88., 178., 135.,  84., 182., 141.]), 5.0]
['418', array([136., 134., 137., 136., 135.,  78., 188., 139., 139., 136.]), 5.0]
['418', array([135., 136., 132., 137.,  83., 183., 140.,  87., 182., 138.]), 5.0]
['418', array([135., 137., 134., 135., 136.,  81., 190., 136., 134., 136.]), 5.0]
['418', array([136., 138.,  78.,  84.,  78., 163., 140.,  80.,  78., 112.]), 5.0]
['418', array([142., 131.,  82., 190., 138., 139., 134., 137., 133., 134.]), 5.0]
['418', array([135.,  87., 187., 138.,  81., 186., 139.,  80., 187., 139.]), 5.0]
['418', array([ 81., 191., 139.,  78., 189., 139.,  88., 186., 139.,  79.]), 5.0]
['418', array([186., 138.,  78., 188., 139.,  81., 188., 142.,  77., 194.]), 5.0]
['418', array([141.,  79., 190., 139.,  80., 187., 139.,  78., 190., 175.]), 5.0]
['418', array([ 41., 192., 139.,  80., 195., 142.,  77., 191., 139.,  78.]), 5.0]
['418', array([1

['419', array([194., 145.,  82., 197., 142.,  87., 190., 145.,  82., 194.]), 5.0]
['419', array([142.,  83., 199., 141.,  82., 190., 142.,  82., 200., 142.]), 5.0]
['419', array([ 83., 194., 145.,  81., 196., 144.,  84., 193., 141.,  83.]), 5.0]
['419', array([196., 140.,  81., 200., 143.,  82., 195., 142.,  82., 199.]), 5.0]
['419', array([140.,  82., 197., 143.,  83., 199., 144., 141.,  89.,  78.]), 5.0]
['419', array([ 79., 169., 145.,  82., 195., 143.,  79., 195., 141.,  82.]), 5.0]
['419', array([ 45., 163., 139.,  83.,  85., 157.,  74.,  77., 114., 103.]), 5.0]
['419', array([151., 140., 141., 142., 140., 142., 136., 140., 138., 141.]), 5.0]
['419', array([143., 140., 140., 138.,  86., 192., 140.,  82., 200., 140.]), 5.0]
['419', array([ 82., 191., 143.,  81., 203., 143.,  76.,  77.,  72.,  96.]), 5.0]
['419', array([ 80., 196., 141., 145.,  88., 191., 139.,  80., 198., 141.]), 5.0]
['419', array([141., 141., 139., 143., 144.,  93., 130., 193., 142.,  80.]), 5.0]
['419', array([1

['419', array([110.,  50., 100.,  90., 100., 110., 100., 150., 140., 110.]), 13.0]
['419', array([220., 160., 140., 120., 190., 180., 170., 170., 130., 170.]), 13.0]
['419', array([180., 160., 100.,  90., 160., 160., 160.,  40., 100., 200.]), 13.0]
['419', array([150.,  90.,  40., 200., 170., 170., 120.,  70., 130.,  80.]), 13.0]
['419', array([ 90., 170., 160., 100.,  30., 200., 170.,  90., 100., 130.]), 13.0]
['419', array([170., 160.,  90.,  40.,  60.,  70.,  70., 130., 160., 100.]), 13.0]
['419', array([ 60., 140., 120.,  50., 110.,  50., 100.,  60., 100.,  50.]), 13.0]
['419', array([100., 100., 110., 100.,  30., 110., 240., 190., 170., 130.]), 13.0]
['419', array([ 60.,  30., 100., 220., 170.,  90., 100.,  30., 120., 150.]), 13.0]
['419', array([130., 200., 110.,  50., 130., 100.,  80., 140., 210., 180.]), 13.0]
['420', array([231., 229., 231., 228., 232., 227., 226., 232., 226., 226.]), 5.0]
['420', array([224., 230., 233., 180., 264., 234., 233., 238., 226., 227.]), 5.0]
['420'

['421', array([ 65.,  65.,  76., 110.,  80., 208., 145., 150., 138.,  63.]), 5.0]
['421', array([124.,  91., 141., 136., 139., 185., 115.,  40.,  84.,  47.]), 5.0]
['421', array([ 85.,  95.,  62.,  96., 165., 139., 116., 162.,  94.,  42.]), 5.0]
['421', array([139., 276., 140., 137.,  88.,  33., 144.,  49.,  89., 100.]), 5.0]
['421', array([ 48., 133., 141., 135., 148., 124.,  66., 204., 201.,  67.]), 5.0]
['421', array([119., 158., 134., 159., 114., 154., 174.,  88., 132., 157.]), 5.0]
['421', array([ 50.,  43., 159., 117., 156., 135., 137., 140., 139., 134.]), 5.0]
['421', array([141., 135., 133., 145., 141., 138., 137., 138., 138., 140.]), 5.0]
['421', array([138., 137., 139., 135., 139., 138., 138., 135., 136.,  93.]), 5.0]
['421', array([185., 138., 137., 157., 115., 280., 137., 139., 119.,  86.]), 5.0]
['421', array([ 70., 137., 136., 140., 137., 139., 141., 140., 139., 140.]), 5.0]
['421', array([137., 140., 141., 139., 138., 138., 139., 140., 143., 139.]), 5.0]
['421', array([1

['422', array([147., 147., 152., 147., 145., 147., 143., 149., 146., 147.]), 5.0]
['422', array([146., 142., 144., 146., 144., 145., 143., 148., 147., 142.]), 5.0]
['422', array([145., 144., 148., 148., 141., 144., 144., 143., 147., 146.]), 5.0]
['422', array([143., 145., 142., 144., 145., 140., 145., 146., 143., 146.]), 5.0]
['422', array([141., 143., 143., 141., 145., 149., 145., 143., 144., 144.]), 5.0]
['422', array([146., 148., 144., 145., 145., 147., 146., 145., 147., 146.]), 5.0]
['422', array([149., 149., 145., 146., 151., 150., 148., 150., 144., 128.]), 5.0]
['422', array([168., 147., 152., 148., 146., 145., 147., 149., 146., 148.]), 5.0]
['422', array([148., 149., 149., 146., 149., 146., 148., 151., 148., 148.]), 5.0]
['422', array([145., 148., 150., 148., 146., 145., 146., 148., 148., 148.]), 5.0]
['422', array([146., 149., 151., 144., 148., 143., 147., 151., 145., 147.]), 5.0]
['422', array([143., 147., 149., 145., 149., 148., 145., 147., 147., 144.]), 5.0]
['422', array([1

...

['610', array([ 92., 206., 318., 217., 354., 346., 341., 339., 333., 335.]), 5.0]
['610', array([324., 333., 330., 281., 327., 335., 184., 186., 327., 346.]), 5.0]
['610', array([336., 343., 333., 337., 332., 303., 334., 329., 337., 326.]), 5.0]
['610', array([313., 301., 215., 322., 336., 327., 333., 255., 331., 290.]), 5.0]
['610', array([325., 265., 327., 320., 288., 326., 328., 323., 189., 295.]), 5.0]
['610', array([212., 329., 192.,  97., 230., 284.,  87., 352., 197., 200.]), 4.0]
['610', array([252., 333., 200., 341., 182.,  98.,  91., 287., 148., 119.]), 4.0]
['610', array([386., 196., 223., 190., 352., 268., 160., 326., 181., 224.]), 4.0]
['610', array([189., 191., 313., 183., 107., 402., 193., 203., 317., 350.]), 4.0]
['610', array([221., 344., 204.,  96., 247., 200., 248., 214., 346., 296.]), 4.0]
['610', array([374., 192., 320., 185., 102., 399., 490., 183., 509., 195.]), 4.0]
['610', array([281., 372., 205., 113., 118., 377., 245.,  75.,  80.,  87.]), 4.0]
['610', array([ 

['611', array([160., 150., 160., 160., 160., 150., 160., 160., 160., 150.]), 10.0]
['611', array([160., 150., 160., 160., 160., 160., 150., 160., 160., 160.]), 10.0]
['611', array([150., 160., 150., 160., 150., 160., 160., 150., 150., 160.]), 10.0]
['611', array([160., 160., 150., 160., 150., 160., 150., 160., 160., 150.]), 10.0]
['611', array([160., 160., 150., 160., 150., 160., 160., 150., 160., 160.]), 10.0]
['611', array([150., 160., 160., 160., 150., 160., 150., 170., 150., 160.]), 10.0]
['611', array([160., 160., 150., 160., 160., 160., 150., 160., 160., 160.]), 10.0]
['611', array([160., 160., 160., 150., 160., 160., 160., 150., 160., 160.]), 10.0]
['611', array([160., 150., 160., 160., 160., 150., 160., 160., 160., 150.]), 10.0]
['611', array([160., 160., 160., 150., 160., 160., 150., 160., 160., 160.]), 10.0]
['611', array([160., 150., 160., 150., 150., 160., 160., 160., 160., 150.]), 10.0]
['611', array([160., 150., 160., 160., 160., 150., 160., 160., 160., 150.]), 10.0]
['61

['611', array([110., 110., 110., 110., 110., 110., 100., 110., 110., 110.]), 14.0]
['611', array([110., 110., 100., 110., 110., 110., 110., 110., 110., 110.]), 14.0]
['611', array([100., 110., 110., 110., 110., 110., 100., 110., 110., 110.]), 14.0]
['611', array([110., 110., 100., 110., 110., 110., 110., 110., 100., 110.]), 14.0]
['611', array([110., 110., 110., 100., 110., 110., 110., 110., 110., 100.]), 14.0]
['611', array([110., 110., 110., 110., 110., 100., 110., 110., 110., 110.]), 14.0]
['611', array([110., 110., 110., 110., 100., 110., 110., 110., 110., 110.]), 14.0]
['611', array([110., 100., 110., 110., 110., 110., 110., 110., 100., 110.]), 14.0]
['611', array([110., 120., 100., 110., 110., 110., 110., 100., 110., 110.]), 14.0]
['611', array([110., 110., 110., 110., 100., 110., 110., 110., 110., 100.]), 14.0]
['611', array([110., 110., 110., 110., 110., 110., 100., 110., 110., 110.]), 14.0]
['611', array([110., 110., 110., 110., 100., 110., 110., 110., 110., 110.]), 14.0]
['61

['612', array([150., 410., 150., 410., 150., 410., 150., 410., 550., 160.]), 5.0]
['612', array([410., 150., 410., 160., 410., 160., 410., 150., 400., 160.]), 5.0]
['612', array([410., 560., 160., 390., 150., 410., 170., 380., 150., 410.]), 5.0]
['612', array([160., 410., 150., 410., 150., 400., 150., 390., 150., 410.]), 5.0]
['612', array([150., 400., 160., 410., 150., 410., 150., 410., 150., 400.]), 5.0]
['612', array([160., 410., 150., 410., 150., 410., 150., 410., 150., 420.]), 5.0]
['612', array([150., 410., 150., 390., 150., 410., 150., 410., 160., 400.]), 5.0]
['612', array([150., 410., 160., 380., 180., 380., 180.,  90., 260., 160.]), 5.0]
['612', array([370., 180.,  90.,  90., 130., 330., 170.,  90.,  90.,  80.]), 5.0]
['612', array([140., 140., 180.,  80.,  90., 350., 180.,  80., 120.,  50.]), 14.0]
['612', array([100., 130., 280., 170., 160., 170., 180., 190., 100., 260.]), 14.0]
['612', array([ 40., 170., 160., 120., 170., 100.,  40.,  80., 130.,  80.]), 14.0]
['612', array

['614', array([ 60.,  90.,  60.,  60.,  60., 140.,  50.,  60., 110.,  60.]), 14.0]
['614', array([ 60., 140.,  30., 100.,  80.,  70.,  70., 110.,  50.,  60.]), 14.0]
['614', array([150.,  50.,  80.,  30., 150., 110.,  60.,  40., 170., 100.]), 14.0]
['614', array([ 50.,  50., 100.,  60.,  80., 130.,  50., 110., 200.,  60.]), 14.0]
['614', array([ 50.,  50., 100., 110., 100., 110., 160.,  50., 150., 140.]), 14.0]
['614', array([ 80., 100.,  60., 150., 100.,  50.,  60.,  80., 130.,  90.]), 14.0]
['614', array([ 70., 170.,  50., 110.,  60., 150.,  50., 200., 160.,  60.]), 14.0]
['614', array([ 80.,  60.,  60.,  60., 120., 110., 140.,  30., 120.,  60.]), 14.0]
['614', array([ 90.,  70., 180.,  90., 140., 110., 100., 140., 180., 240.]), 14.0]
['614', array([ 50.,  50.,  60.,  60.,  70.,  70.,  90., 110.,  80.,  60.]), 14.0]
['614', array([ 90., 110.,  30., 110.,  60.,  50.,  60.,  50., 130., 100.]), 14.0]
['614', array([ 30., 110.,  90., 160., 120., 110.,  60.,  70.,  60.,  70.]), 14.0]
['61

In [22]:
record_id='418'
qrs_i = load_qrs_i(record_id)
#print type(qrs_i)
qrs_amp = load_qrs_amp(record_id)
ann = get_annotation(record_id)
annot_positions = get_annotation_positions(ann)
annotated_r_peaks = create_qrs_annotation_mapping(qrs_i, qrs_amp, annot_positions)
annotated_rr_segments = create_rr_annotation(annotated_r_peaks)
print(annotated_rr_segments)

[[2.4000e+01 1.2800e+02 1.0400e+02 5.0000e+00]
 [1.2800e+02 2.6100e+02 1.3300e+02 5.0000e+00]
 [2.6100e+02 3.4500e+02 8.4000e+01 5.0000e+00]
 ...
 [5.2460e+05 5.2473e+05 1.3000e+02 5.0000e+00]
 [5.2473e+05 5.2486e+05 1.3000e+02 5.0000e+00]
 [5.2486e+05 5.2499e+05 1.3000e+02 5.0000e+00]]


In [30]:
# for every record print label episode!
for record in VFDB_RECORDS:
    record_id=str(record)
    record_data=get_record(record_id)
    record_size=record_data.sig_len
    ann = get_annotation(record_id)
    annot_positions = get_annotation_positions(ann)
    annot_positions = [(0, '(N\x00')] + annot_positions
    print ('record:', record_id)
    i=0
    while i<len(annot_positions):
        if i==0: start=i
        else: start=annot_positions[i][0] + 1
        if i==len(annot_positions)-1:
            end=record_size-1
        else:
            end=annot_positions[i+1][0]
        print (start,end,annot_positions[i][1])
        i+=1
        
    

record: 418
0 18 (N 
19 99624 (N 
99625 101499 (VFL 
101500 133092 (N 
133093 134038 (VFL 
134039 135775 (N 
135776 136628 (VFL 
136629 153057 (N 
153058 154115 (VFL 
154116 154942 (N 
154943 156291 (VFL 
156292 159442 (N 
159443 160516 (VFL 
160517 169192 (N 
169193 169807 (VFL 
169808 173054 (N 
173055 173673 (VFL 
173674 174788 (N 
174789 175403 (VFL 
175404 176259 (N 
176260 177868 (VFL 
177869 190080 (N 
190081 191249 (VFL 
191250 191807 (N 
191808 192695 (VFL 
192696 195631 (N 
195632 196794 (VFL 
196795 200211 (N 
200212 200634 (VFL 
200635 216788 (N 
216789 219038 (VFL 
219039 219961 (N 
219962 224019 (VFL 
224020 225355 (N 
225356 226057 (VFL 
226058 227211 (N 
227212 229269 (VFL 
229270 231310 (N 
231311 232724 (VFL 
232725 234499 (N 
234500 235538 (VFL 
235539 254230 (N 
254231 255365 (VFL 
255366 256019 (N 
256020 256884 (VFL 
256885 257249 (N 
257250 257980 (VFL 
257981 259557 (N 
259558 261903 (VFL 
261904 262749 (N 
262750 263519 (VFL 
263520 269307 (N 
269308 270999 (VF

record: 430
0 28 (N 
29 51903 (BI 
51904 53211 (VT 
53212 85615 (BI 
85616 93134 (NOISE 
93135 94423 (BI 
94424 94903 (VT 
94904 98596 (BI 
98597 99557 (BI 
99558 118057 (VFL 
118058 120807 (NOISE 
120808 131153 (VF 
131154 134519 (ASYS 
134520 138076 (SBR 
138077 156576 (HGEA 
156577 157923 (VT 
157924 163192 (HGEA 
163193 169179 (VT 
169180 170237 (ASYS 
170238 172275 (HGEA 
172276 173891 (VT 
173892 175429 (HGEA 
175430 193891 (VF 
193892 196217 (ASYS 
196218 273294 (VF 
273295 274391 (ASYS 
274392 310544 (VF 
310545 315333 (ASYS 
315334 319871 (SBR 
319872 406794 (VF 
406795 407102 (ASYS 
407103 415025 (VER 
415026 489621 (VT 
489622 493794 (VER 
493795 524999 (VF 
record: 602
0 64 (N 
65 121275 (PM 
121276 121756 (VT 
121757 286814 (PM 
286815 287333 (VT 
287334 289352 (PM 
289353 289910 (VT 
289911 311833 (PM 
311834 408525 (VT 
408526 411044 (ASYS 
411045 422487 (N 
422488 467987 (NOISE 
467988 524999 (PM 
record: 605
0 64 (N 
65 411942 (NOISE 
411943 419115 (VT 
419116 524999 (

In [31]:
#print the number of R in each episode
for record in VFDB_RECORDS:
    record_id=str(record)
    print ('record:', record_id)
    qrs_i = load_qrs_i(record_id)
    #print type(qrs_i)
    qrs_amp = load_qrs_amp(record_id)
    ann = get_annotation(record_id)
    annot_positions = get_annotation_positions(ann)
    annotated_r_peaks = create_qrs_annotation_mapping(qrs_i, qrs_amp, annot_positions)
    i=1
    annot=annotated_r_peaks[0][2]
    s=1
    while i<annotated_r_peaks[:,0].size:
        if annotated_r_peaks[i-1][2]==annotated_r_peaks[i][2]:
            s+=1
        else:
            print(ann_num_to_str[annotated_r_peaks[i-1][2]], '\t', s)
            s=1
        i+=1

record: 418
(N  	 748
(VFL  	 22
(N  	 240
(VFL  	 12
(N  	 17
(VFL  	 11
(N  	 128
(VFL  	 11
(N  	 9
(VFL  	 17
(N  	 26
(VFL  	 11
(N  	 69
(VFL  	 5
(N  	 26
(VFL  	 8
(N  	 9
(VFL  	 6
(N  	 7
(VFL  	 21
(N  	 96
(VFL  	 16
(N  	 4
(VFL  	 12
(N  	 23
(VFL  	 11
(N  	 26
(VFL  	 3
(N  	 136
(VFL  	 22
(N  	 6
(VFL  	 48
(N  	 11
(VFL  	 7
(N  	 9
(VFL  	 23
(N  	 17
(VFL  	 17
(N  	 14
(VFL  	 12
(N  	 152
(VFL  	 13
(N  	 5
(VFL  	 11
(N  	 4
(VFL  	 11
(N  	 12
(VFL  	 25
(N  	 7
(VFL  	 7
(N  	 46
(VFL  	 25
(N  	 3
(VFL  	 1
(N  	 4
(VFL  	 4
(N  	 11
(VFL  	 26
(N  	 28
(VFL  	 23
(N  	 61
(VFL  	 23
(N  	 70
(VFL  	 8
(N  	 3
(VFL  	 2
(N  	 5
(VFL  	 3
(N  	 3
(VFL  	 4
(N  	 2
(VFL  	 12
(N  	 52
(VFL  	 4
(N  	 2
(VFL  	 17
(N  	 2
(VFL  	 6
(N  	 3
(VFL  	 27
(N  	 4
(VFL  	 3
(N  	 9
(VFL  	 12
(N  	 59
(VFL  	 18
(N  	 3
(VFL  	 3
(N  	 35
(VFL  	 5
(N  	 43
(VFL  	 3
(N  	 51
(VFL  	 4
(N  	 3
(VFL  	 3
(N  	 75
(VFL  	 8
(N  	 33
(VFL  	 5
(N  	 35
(VFL  	 4
(N  	 3
